In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from os.path import join as pjoin
from natsort import natsorted
import statsmodels.api as sm
from statsmodels.formula.api import ols

sys.path.append("../")
import circletrack_behavior as ctb
import plotting_functions as pf

In [33]:
## Settings
dpath = './intermediate_data' 
fig_path = './intermediate_figures'

chance_color = 'darkgrey'
avg_color = '#00802D'
exp_color_dict = {'Inhibition1': 'darkgrey', 'mcr1': 'blue', 'mcr2': 'midnightblue', 'mcr3': 'orchid', 
                  'mcr4': 'darkorchid', 'mcls3': 'darkgreen', 'mcls4': 'slateblue'}

session_list = [f'A{x}' for x in np.arange(1, 6)]

if not os.path.exists(fig_path):
    os.makedirs(fig_path)

## Load previously-saved data
ct_df = pd.read_feather(pjoin(dpath, 'ct_df.feat'))
probe_df = pd.read_feather(pjoin(dpath, 'probe_df.feat'))

## Select only young adult mice (age is in months)
ct_df = ct_df[ct_df['age'] <= 6]
probe_df = probe_df[probe_df['age'] <= 6]

### Circle track rewards and lick accuracy across the first five days.

In [45]:
## Plot 5th lick accuracy across days
fig = pf.plot_behavior_across_days(ct_df, x_var='day', y_var='lick_accuracy', groupby_var=['day'], plot_transitions=None,
                                   marker_color=avg_color, avg_color=avg_color, expert_line=False, chance=True, symbols=['circle'],
                                   x_title='Day', y_title='Lick Accuracy (%)', titles=[''], 
                                   plot_datapoints=False, height=500, width=500)
fig.update_yaxes(range=[0, 100])
fig.show()
fig.write_image(pjoin(fig_path, 'lick_accuracy_across_days.png'), width=500, height=500)

In [46]:
## Plot rewards across days
fig = pf.plot_behavior_across_days(ct_df, x_var='day', y_var='rewards', groupby_var=['day'], plot_transitions=None,
                                   marker_color=avg_color, avg_color=avg_color, expert_line=False, chance=False, symbols=['circle'],
                                   x_title='Day', y_title='Rewards', titles=[''], 
                                   plot_datapoints=False, height=500, width=500)
fig.update_yaxes(range=[0, 100])
fig.show()
fig.write_image(pjoin(fig_path, 'rewards_across_days.png'), width=500, height=500)

In [5]:
## Plot each experiment as separate colors
fig = pf.custom_graph_template(x_title='Day', y_title='', rows=1, columns=2, height=500, width=1000, 
                               shared_x=True, titles=[''])
for exp in ct_df['experiment'].unique():
    exp_data = ct_df[ct_df['experiment'] == exp].reset_index(drop=True)
    avg = exp_data.groupby(['day'], as_index=False).agg({'lick_accuracy': ['mean', 'sem'], 'rewards': ['mean', 'sem']})
    fig.add_trace(go.Scattergl(x=avg['day'], y=avg['lick_accuracy']['mean'], mode='lines+markers', line_color=exp_color_dict[exp],
                               error_y=dict(type='data', array=avg['lick_accuracy']['sem']), name=exp,
                               legendgroup=exp, showlegend=False), row=1, col=1)
    fig.add_trace(go.Scattergl(x=avg['day'], y=avg['rewards']['mean'], mode='lines+markers', line_color=exp_color_dict[exp],
                               error_y=dict(type='data', array=avg['rewards']['sem']), name=exp,
                               legendgroup=exp), row=1, col=2)
fig.update_yaxes(title='Lick Accuracy (%)', col=1, range=[0, 100])
fig.update_yaxes(title='Rewards', col=2)
fig.update_layout(legend=dict(orientation='h', y=1.30))
fig.show()
fig.write_image(pjoin(fig_path, 'fifth_lick_acc_across_experiments.png'))

In [6]:
## Correlate first lick accuracy and fifth lick accuracy
fig = pf.custom_graph_template(x_title='Fifth Lick Accuracy (%)', y_title='First Lick Accuracy (%)')
fig.add_trace(go.Scattergl(x=ct_df['lick_accuracy'], y=ct_df['first_lick_accuracy'], mode='markers',
                           marker=dict(line=dict(width=1), color='darkgrey'), opacity=0.7))
fig.update_yaxes(range=[0, 100])
fig.add_hline(y=25, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.add_vline(x=25, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.show()
fig.write_image(pjoin(fig_path, 'fifth_lick_vs_first_lick_acc.png'))

In [7]:
## Correlate fifth lick accuracy with reward separated by day
fig = pf.custom_graph_template(x_title='Rewards', y_title='', rows=1, columns=5,
                               titles=session_list, shared_x=True, shared_y=True, width=1000)
for session in session_list:
    sdata = ct_df[ct_df['session'] == session]
    fig.add_trace(go.Scattergl(x=sdata['rewards'], y=sdata['lick_accuracy'], mode='markers', showlegend=False,
                               marker=dict(line=dict(width=1), color='darkgrey'), opacity=0.7), row=1, col=int(session[-1]))
    fig.add_trace(go.Scattergl(x=[np.mean(sdata['rewards'])], y=[np.mean(sdata['lick_accuracy'])], mode='markers', marker_color='red',
                               marker=dict(line=dict(width=2), size=8), showlegend=False), row=1, col=int(session[-1]))
fig.update_yaxes(range=[0, 100])
fig.add_hline(y=25, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.update_yaxes(title='Fifth Lick Accuracy (%)', col=1)
fig.show()
fig.write_image(pjoin(fig_path, 'fifth_lick_accuracy_correlated_rewards_each_day.png'))

In [8]:
## Correlate first lick accuracy with reward separated by day
fig = pf.custom_graph_template(x_title='Rewards', y_title='', rows=1, columns=5,
                               titles=session_list, shared_x=True, shared_y=True, width=1000)
for session in session_list:
    sdata = ct_df[ct_df['session'] == session]
    fig.add_trace(go.Scattergl(x=sdata['rewards'], y=sdata['first_lick_accuracy'], mode='markers', showlegend=False,
                               marker=dict(line=dict(width=1), color='darkgrey'), opacity=0.7), row=1, col=int(session[-1]))
    fig.add_trace(go.Scattergl(x=[np.mean(sdata['rewards'])], y=[np.mean(sdata['first_lick_accuracy'])], mode='markers', marker_color='red',
                               marker=dict(line=dict(width=2), size=8), showlegend=False), row=1, col=int(session[-1]))
fig.update_yaxes(range=[0, 100])
fig.add_hline(y=25, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.update_yaxes(title='First Lick Accuracy (%)', col=1)
fig.show()

In [36]:
## Distribution of number of trials ran each day
fig = pf.custom_graph_template(x_title='Trials', y_title='', rows=1, columns=5,
                               titles=session_list, shared_x=True, shared_y=True, width=1000)
for session in session_list:
    sdata = ct_df[ct_df['session'] == session]
    fig.add_trace(go.Histogram(x=sdata['num_trials'], marker_color=avg_color, opacity=0.7, showlegend=False,
                               marker_line_width=2, marker_line_color='black'), row=1, col=int(session[-1]))
    fig.add_vline(x=np.round(np.mean(sdata['num_trials'])), line_width=1, line_dash='dash', opacity=1, line_color='red', col=int(session[-1]))
fig.update_yaxes(title='Count', col=1)
fig.show()
fig.write_image(pjoin(fig_path, 'num_trials_histogram_each_day.png'))

In [49]:
## Plot probe accuracy on day 5
last_day = probe_df[probe_df['day'] == 5] ## select the last day in A
last_day = last_day.astype({'day': str})
avg_probe = last_day.groupby(['day'], as_index=False).agg({'probe_acc': ['mean', 'sem']})
fig = pf.custom_graph_template(x_title='Day', y_title='Lick Accuracy (%)', titles=['Probe Accuracy'])
fig.add_trace(go.Scattergl(x=avg_probe['day'], y=avg_probe['probe_acc']['mean'], mode='markers', marker_color=avg_color,
                           error_y=dict(type='data', array=avg_probe['probe_acc']['sem']), showlegend=False))
fig.add_hline(y=25, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.update_yaxes(range=[0, 100])
fig.show()
fig.write_image(pjoin(fig_path, 'probe_accuracy.png'))

In [47]:
## Plot probe accuracy on day 5 as a boxplot
last_day = probe_df[probe_df['day'] == 5] ## select the last day in A
last_day = last_day.astype({'day': str})
fig = pf.custom_graph_template(x_title='Day', y_title='Lick Accuracy (%)', titles=[''], width=400)
fig.add_trace(go.Box(x=last_day['day'], y=last_day['probe_acc'], marker_color=avg_color, showlegend=False,
                     boxpoints='all', jitter=0.8, pointpos=-2, opacity=0.7))

fig.add_hline(y=25, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.update_yaxes(range=[-2, 102])
fig.show()
fig.write_image(pjoin(fig_path, 'probe_acc_boxplot.png'), width=400, height=500)

In [ ]:
## Plot the distribution of probe accuracy on day 5
fig = pf.custom_graph_template(x_title='Lick Accuracy (%)', y_title='Count', titles=['Probe Accuracy'])
fig.add_trace(go.Histogram(x=last_day['probe_acc'], marker_line_width=2, marker_line_color='black', marker_color='midnightblue'))
fig.add_vline(x=25, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.show()
fig.write_image(pjoin(fig_path, 'probe_acc_distribution.png'))

In [50]:
## Correlate probe accuracy with fifth lick accuracy in the sessions before
fig = pf.custom_graph_template(x_title='Lick Accuracy', y_title='', width=1200,
                               titles=session_list, shared_y=True, shared_x=True, rows=1, columns=5)
for day in [1, 2, 3, 4, 5]:
    day_data = ct_df[ct_df['day'] == day].reset_index(drop=True)
    plot_data = pd.merge(day_data, last_day, on='mouse')
    fig.add_trace(go.Scattergl(x=plot_data['lick_accuracy'], y=plot_data['probe_acc'], mode='markers',
                            marker=dict(line=dict(width=1), color='darkgrey'), opacity=0.7, showlegend=False), row=1, col=day)
fig.add_hline(y=25, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.add_vline(x=25, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.update_yaxes(title='Probe Accuracy (%)', col=1)
fig.show()

In [54]:
## Correlate probe accuracy with the number of trials in the sessions before
fig = pf.custom_graph_template(x_title='Trials', y_title='', width=1000,
                               titles=session_list, shared_y=True, shared_x=True, rows=1, columns=5)
for day in [1, 2, 3, 4, 5]:
    day_data = ct_df[ct_df['day'] == day].reset_index(drop=True)
    plot_data = pd.merge(day_data, last_day, on='mouse')
    fig.add_trace(go.Scattergl(x=plot_data['num_trials'], y=plot_data['probe_acc'], mode='markers',
                            marker=dict(line=dict(width=1), color='darkgrey'), opacity=0.7, showlegend=False), row=1, col=day)
fig.add_hline(y=25, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.update_yaxes(title='Probe Accuracy (%)', col=1)
fig.show()

In [53]:
## Plot number of trials vs number of rewards
fig = pf.custom_graph_template(x_title='Trials', y_title='', rows=1, columns=5,
                               titles=session_list, shared_x=True, shared_y=True, width=1000)
for session in session_list:
    sdata = ct_df[ct_df['session'] == session]
    fig.add_trace(go.Scattergl(x=sdata['num_trials'], y=sdata['rewards'], mode='markers', showlegend=False,
                               marker=dict(line=dict(width=1), color='darkgrey'), opacity=0.7), row=1, col=int(session[-1]))
fig.update_yaxes(title='Rewards', col=1)
fig.show()